In [2]:
import mujoco
import os
import jax
from jax import numpy as jnp
import numpy as np
# from flax.training import orbax_utils
# from flax import struct
from matplotlib import pyplot as plt
# import mediapy as media
# from orbax import checkpoint as ocp

import mujoco

In [12]:
xml = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

In [26]:
## interesting functions

# model.geom('green_sphere').rgba
id = mujoco.mj_name2id(model, mujoco.mjtObj.mjOBJ_GEOM, 'green_sphere')
# model.geom_rgba
model.body('world') ## is the same as model.body(0)

1


In [35]:
data = mujoco.MjData(model)

In [39]:
# print positions of data

print('raw access:\n', data.geom_xpos)
# you can also access indivial parts by name
print('\nnamed access\n', data.geom('green_sphere').xpos)

mujoco.mj_kinematics(model, data) # mjData properties are 'propagated'



raw access:
 [[0. 0. 0.]
 [0. 0. 0.]]

named access
 [0. 0. 0.]
